In [1]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing as preproc
from scipy import stats
import numpy as np

In [2]:
df=pd.read_parquet("diabetic_data_cleaned_pca_som_ae.parquet")

In [3]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,ae_1,ae_2,ae_3,ae_4,ae_5,ae_6,ae_7,ae_8,ae_9,ae_10
0,Caucasian,Female,[10-20),1,1,7,3,NaN,NaN,59,...,13.208113,32.464939,31.501293,17.181660,31.385792,22.330921,31.645599,31.576422,20.440641,32.343941
1,AfricanAmerican,Female,[20-30),1,1,7,2,NaN,NaN,11,...,2.133152,26.914845,29.317085,6.315910,26.429140,22.126324,14.199553,32.529652,19.284925,9.772506
2,Caucasian,Male,[30-40),1,1,7,2,NaN,NaN,44,...,14.186540,24.076536,16.652319,10.141067,25.860373,13.117648,38.765411,28.151123,15.377409,25.999193
3,Caucasian,Male,[40-50),1,1,7,1,NaN,NaN,51,...,21.980705,20.161404,16.131815,11.691567,27.503544,26.599667,21.637993,43.432602,40.661266,24.253494
4,Caucasian,Male,[50-60),2,1,2,3,NaN,NaN,31,...,30.563612,12.413562,14.935289,15.416083,3.050638,8.946961,35.092857,28.619825,17.472162,16.211063


In [4]:
df.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'metformin', 'repaglinide', 'glimepiride',
       'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin',
       'glyburide-metformin', 'change', 'diabetesMed', 'readmitted', 'PC1',
       'PC2', 'PC3', 'som_categoricals_jaccard_0',
       'som_categoricals_jaccard_1', 'som_jaccard_0', 'som_jaccard_1',
       'som_euc_0', 'som_euc_1', 'ae_0', 'ae_1', 'ae_2', 'ae_3', 'ae_4',
       'ae_5', 'ae_6', 'ae_7', 'ae_8', 'ae_9', 'ae_10'],
      dtype='object')

In [5]:
DB_COLUMNS = [
  'PC1','PC2', 'PC3','ae_0', 'ae_1', 'ae_2', 'ae_3', 'ae_4',
       'ae_5', 'ae_6', 'ae_7', 'ae_8', 'ae_9', 'ae_10'
]

In [6]:
df[DB_COLUMNS]

,PC1,PC2,PC3,ae_0,ae_1,ae_2,ae_3,ae_4,ae_5,ae_6,ae_7,ae_8,ae_9,ae_10
0,-0.214843,0.070744,-0.194516,16.559961,13.208113,32.464939,31.501293,17.181660,31.385792,22.330921,31.645599,31.576422,20.440641,32.343941
1,0.417941,-0.522845,0.160733,17.015982,2.133152,26.914845,29.317085,6.315910,26.429140,22.126324,14.199553,32.529652,19.284925,9.772506
2,-0.129162,-0.131116,-0.050765,13.954587,14.186540,24.076536,16.652319,10.141067,25.860373,13.117648,38.765411,28.151123,15.377409,25.999193
3,-0.333245,-0.143083,-0.009183,21.054396,21.980705,20.161404,16.131815,11.691567,27.503544,26.599667,21.637993,43.432602,40.661266,24.253494
4,0.640729,-0.446888,-0.059434,12.781404,30.563612,12.413562,14.935289,15.416083,3.050638,8.946961,35.092857,28.619825,17.472162,16.211063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100239,-0.226515,0.049629,-0.151316,16.426741,21.162048,21.151413,13.932247,13.524100,17.923170,12.035247,0.964530,17.563910,25.211979,24.368067
100240,0.266458,-0.086568,-0.008378,14.752852,6.173789,12.620147,19.054213,6.410842,12.630477,9.950077,14.461571,4.194087,13.613973,11.133177
100241,-0.277035,-0.027778,-0.406056,15.978240,15.544564,19.770901,18.339804,24.189186,27.886076,18.692892,2.756591,32.498947,36.339996,39.467510
100242,0.291241,0.333055,0.077571,21.903273,42.385860,22.286873,16.340101,18.980680,25.503567,24.807924,18.306873,22.323936,14.109750,21.605391


In [7]:
X=df[DB_COLUMNS]

In [8]:
# Scale the independent variables
#scaler = StandardScaler()
X_scaled = preproc.MinMaxScaler().fit_transform(X)

In [9]:
# Apply DBSCAN
epsilon = 0.34 # You may need to adjust this parameter
min_samples = 10 # You may need to adjust this parameter
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
#dbscan.fit(X_scaled)
dbscan.fit(X_scaled)


DBSCAN(eps=0.34, min_samples=10)

In [10]:
# Extract outlier labels
outlier_mask = dbscan.labels_ == -1

# Get the outliers
outliers = X[outlier_mask]

print(len(outliers))

1140


In [11]:
len(set(dbscan.labels_))

3

In [12]:
# Analyze outliers
print(len(df[outlier_mask]))

1140


In [13]:
non_outlier_mask=dbscan.labels_ != -1

In [14]:
print(len(df[non_outlier_mask]))

99104


In [15]:
def display_df(df: pd.DataFrame, title: str = ""):
  display(df.style.hide(axis="index").set_caption(title))

In [16]:
TGT = "readmitted"

In [17]:
prop = df.loc[:, TGT].value_counts().reset_index()
display_df(prop.sort_values("readmitted", ascending = False), "full dataset")

prop = df.loc[list(outlier_mask), TGT].value_counts().reset_index()
display_df(prop.sort_values("readmitted", ascending = False), "DBscan outliers")

readmitted,count
NO,53821
>30,35173
<30,11250


readmitted,count
NO,594
>30,419
<30,127


In [18]:
stats.chi2_contingency(np.array([[54864, 35545 , 11357], [594, 419 , 127]]))

Chi2ContingencyResult(statistic=1.7590173860847214, pvalue=0.4149867474808733, dof=2, expected_freq=array([[54843.63232465, 35565.58824558, 11356.77942977],
       [  614.36767535,   398.41175442,   127.22057023]]))

In [19]:
output_df=df[non_outlier_mask]

In [20]:
output_df.to_csv("diabetic_data_cleaned_pca_som_ae_dbscan.csv")
output_df.to_parquet("diabetic_data_cleaned_pca_som_ae_dbscan.parquet")